In [17]:
# Imports
from strkernel.mismatch_kernel import MismatchKernel
from strkernel.mismatch_kernel import preprocess

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [18]:
# Functions
#Get integers from chirality string
def toInt(chirality_string):
    buff = re.split(r"[();]",chirality_string)
    return [int(buff[1]),int(buff[2])]

#Get the chirality from the list of chirality strings
def getChirality(chirality_list):
    return np.array([toInt(i) for i in chirality_list])

In [6]:
# Load the data
training_set = pd.read_csv('../Data/training_set.csv')

In [78]:
# Change labels to 0 and 1
training_set['Label'] = training_set['Label'].apply(lambda x: 1 if x == 'Y' else 0)
# Get the chirality
chirality = getChirality(training_set['Chirality'])
# Dataframe with the chirality
chirality_df = pd.DataFrame(chirality,columns=['m','n'])

In [ ]:
# Preprocess
x = preprocess(training_set['Sequence'])
# Compute mismatch kernels
kernels = MismatchKernel( l = 4,
                          k = 5,
                          m = 1 ).get_kernel(x).kernel

In [83]:
kernels_df = pd.DataFrame(kernels)
#Merge the dataframes
data = pd.concat([kernels_df,chirality_df,training_set['Label']],axis=1)
data.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,965,966,967,968,969,970,971,m,n,Label
532,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,...,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,9,6,0
92,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,0.790000,...,0.318198,0.318198,0.318198,0.318198,0.318198,0.318198,0.318198,11,2,0
721,0.730514,0.730514,0.730514,0.730514,0.730514,0.730514,0.730514,0.730514,0.730514,0.730514,...,0.062994,0.062994,0.062994,0.062994,0.062994,0.062994,0.062994,10,3,0
277,0.876708,0.876708,0.876708,0.876708,0.876708,0.876708,0.876708,0.876708,0.876708,0.876708,...,0.145865,0.145865,0.145865,0.145865,0.145865,0.145865,0.145865,10,3,0
272,0.471405,0.471405,0.471405,0.471405,0.471405,0.471405,0.471405,0.471405,0.471405,0.471405,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11,2,0
450,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,...,0.294884,0.294884,0.294884,0.294884,0.294884,0.294884,0.294884,7,3,0
768,0.415761,0.415761,0.415761,0.415761,0.415761,0.415761,0.415761,0.415761,0.415761,0.415761,...,0.334077,0.334077,0.334077,0.334077,0.334077,0.334077,0.334077,10,2,0
540,0.484934,0.484934,0.484934,0.484934,0.484934,0.484934,0.484934,0.484934,0.484934,0.484934,...,0.444500,0.444500,0.444500,0.444500,0.444500,0.444500,0.444500,10,2,0
226,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9,7,0
444,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,0.458732,...,0.294884,0.294884,0.294884,0.294884,0.294884,0.294884,0.294884,10,2,0


In [82]:
# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(['Label'],axis=1),
                                                                        data['Label'],
                                                                        test_size=0.2,
                                                                        random_state=42)
# Model
model = SVC(kernel='linear')
# Train
model.fit(X_train,Y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


ValueError: The number of classes has to be greater than one; got 1 class